In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output, curdoc
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange, HoverTool
from bokeh.models.annotations import Title
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
from bokeh.layouts import row, column,layout, gridplot
from bokeh.models.widgets import Slider, TextInput
from bokeh.transform import cumsum
from bokeh.palettes import Category10
import nltk
reset_output()
output_notebook()

Loading BokehJS ...

- Mediante el análisis visual, caracterice las condiciones en toda la ciudad y recomiende cómo deben asignarse los recursos a las 5 horas y 30 horas después del terremoto. Incluya evidencia de los datos para apoyar estas recomendaciones. Considere cómo asignar recursos tales como cuadrillas de caminos, cuadrillas de reparación de alcantarillado, equipos de energía y rescate.

In [8]:
data = pd.read_csv("./YInt.csv",index_col=0, parse_dates=True, infer_datetime_format=True)
data = data.dropna()
data.head()

,location,account,message
time,,,
2020-04-06 00:00:00,Weston,Opportunities2,"Take advantheeseage of theesehese One, theeser..."
2020-04-06 00:00:00,Southton,LazyBCouch,@WatchesThomasBird fork it you're back in he s...
2020-04-06 00:02:00,Southton,______3333_____,Two month left bedoreefore you loose this vibe...
2020-04-06 00:04:00,Broadview,RasoHorse49,"billeeeeer, i miss ytouou !"
2020-04-06 00:07:00,West Parton,CuriousPlateBobbie_Mallon,You obviously need to use rumble! #rumble #toW...


In [3]:
pd.unique(data.index.day)

array([ 6,  7,  8,  9, 10])

In [4]:
pd.unique(data.location.values)

array(['Weston', 'Southton', 'Broadview', 'West Parton', 'Old Town',
       'Terrapin Springs', 'Downtown', 'Southwest', 'Scenic Vista',
       'East Parton', 'Cheddarford', 'Palace Hills', 'Safe Town',
       'Easton', 'Chapparal', 'Northwest', 'Oak Willow', 'Pepper Mill',
       'Wilson Forest', 'UNKNOWN', '<Location with-held due to contract>'],
      dtype=object)

In [5]:
sum(data.location == "UNKNOWN"), sum(data.location == "<Location with-held due to contract>")

(31, 95)

In [6]:
data.shape

(41930, 3)

In [7]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, reset_output, curdoc
from bokeh.embed import file_html
from bokeh.models import LabelSet, Label, ColumnDataSource, FactorRange, HoverTool
from bokeh.models.annotations import Title
from bokeh.transform import factor_cmap
import bokeh.plotting as bp
from bokeh.layouts import row, column,layout, gridplot
from bokeh.models.widgets import Slider, TextInput, RangeSlider, Select
from bokeh.transform import cumsum
from bokeh.palettes import Category10, Category20
import nltk

def histogram(df, col, bins=30):
    edges = []
    hist = []
    largura = max(df.index) - min(df.index)
    h = largura/bins
    aux = min(df.index)
    for i in range(100):
        edges.append(aux)
        hist.append(df.loc[aux:aux + h].count()[col])
        aux = aux + h
    edges.append(aux)
    
    return edges, hist

#Se cargan los datos
data = pd.read_csv("YInt.csv",index_col=0, parse_dates=True, infer_datetime_format=True)
#Distribução geral dos dados
p1 = figure(plot_height=300, plot_width=800, title="Quantidade de tweets por intervalo de tempo para todos os bairros",
           tools="hover,pan,wheel_zoom,box_zoom,reset,save",x_axis_type='datetime')
p1.xaxis.axis_label = "dias registrados"
p1.yaxis.axis_label = "quantidade de tweets"
#quantidade de tweets por bairro
p2 = figure(plot_height=300, plot_width=300, title="Quantidade de tweets por bairro",
           tools="hover,pan,wheel_zoom,box_zoom,reset,save",x_range=(-3.5,3.5), y_range=(-3.5,3.5))
#palavras mais faladas
p3 = figure(plot_height=300, plot_width=800, title="Quantidade de tweets por bairro",
           tools="hover,pan,wheel_zoom,box_zoom,reset,save",x_axis_type='datetime', x_range=p1.x_range)
p3.xaxis.axis_label = "dias registrados"
p3.yaxis.axis_label = "quantidade de tweets"
#hashtag mais frequentes
p4 = figure(plot_height=300, plot_width=300, title="Quantidade de usuarios por bairro",
           tools="hover,pan,wheel_zoom,box_zoom,reset,save",x_range=(-3.5,3.5), y_range=(-3.5,3.5))
names = ["Palace Hills", "Northwest", "Old Town", "Safe Town", "Southwest", "Downtown",
         "Wilson Forest", "Scenic Vista", "Broadview", "Chapparal", "Terrapin Springs",
         "Pepper Mill", "Cheddarford", "Easton", "Weston", "Southton", "Oak Willow",
         "East Parton", "West Parton"]
#Barras de interacción
#s_day = RangeSlider(title="intervalo de dias", start=6, end=10, value=(6, 10), step=1)
#s_range = RangeSlider(title="intervalo de horas", start=0, end=24, value=(0,24), step=1)
select_vec = Select(title="Bairro:", value="Palace Hills", options=names)
#Gráfico 1
edges1, hist1 = histogram(data, "location", bins=100)
source1 = ColumnDataSource(dict(hist=hist1,left=edges1[:-1], right=edges1[1:]))
p1.quad(top="hist", bottom=0, left="left", right="right", color = "blue",
            line_color="white", alpha=0.7, source=source1, legend="tweets")
p1.hover.tooltips = [("data inicial", "@left{%F %T}"),
                    ("data final", "@right{%F %T}"),
                    ("quantidade", "@hist")]
p1.hover.formatters = {'left': 'datetime', 'right': 'datetime'}
p1.hover.mode = "vline"

#Gráfico 2
edges2, hist2 = histogram(data.loc[data.location=="Palace Hills"], "location", bins=100)
source2 = ColumnDataSource(dict(hist=hist2,left=edges2[:-1], right=edges2[1:]))
p3.quad(top="hist", bottom=0, left="left", right="right", color = "red",
            line_color="white", alpha=0.7, source=source2, legend="tweets")
p3.hover.tooltips = [("data inicial", "@left{%F %T}"),
                    ("data final", "@right{%F %T}"),
                    ("quantidade", "@hist")]
p3.hover.formatters = {'left': 'datetime', 'right': 'datetime'}
p3.hover.mode = "vline"

#Gráfico 3
data_aux = data.copy()
data_aux.drop(["account","message"], axis=1)
data_aux["values"] = 1
data_aux = data_aux.groupby("location").sum()
data_aux = data_aux.drop(["UNKNOWN", "<Location with-held due to contract>"])
source3 = ColumnDataSource(dict(data_aux, angle=data_aux["values"]/(data_aux["values"]).sum() * 2*np.pi,
                                  color=Category20[19], location=data_aux.index))
p2.wedge(x=0, y=0, radius=3,start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
         line_color="black", fill_color='color', source=source3)
p2.hover.tooltips = [("bairro", "@location"),
                     ("quantidade", "@values")]

#Gráfico 4
data_aux2 = data.copy()
data_aux2.drop("message", axis=1)
data_aux2["values"] = 1
data_aux2 = data_aux2.groupby(["location", "account"], as_index=False).sum()
data_aux2 = data_aux2.set_index("location")
data_aux2 = data_aux2.drop(["UNKNOWN", "<Location with-held due to contract>"])
data_aux2["values"] = 1
data_aux2 = data_aux2.groupby("location").sum()
source4 = ColumnDataSource(dict(data_aux2, angle=data_aux["values"]/(data_aux["values"]).sum() * 2*np.pi,
                               color=Category20[19], location=data_aux2.index))
p4.wedge(x=0, y=0, radius=3,start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
         line_color="black", fill_color='color', source=source4)
p4.hover.tooltips = [("bairro", "@location"),
                     ("quantidade", "@values")]

def update_data(attrname, old, new):

    #Para obtener los valores actuales
    #day_val = s_day.value
    vec_val = select_vec.value
    #hr_val = s_range.value
    #Actualizar grafico 3
    edges2, hist2 = histogram(data.loc[data.location==vec_val], "location", bins=100)
    source2.data = dict(hist=hist2,left=edges2[:-1], right=edges2[1:])
    
    
#Para hacer las actualizaciones
for w in [select_vec]:
    w.on_change('value', update_data)

col_1 = column([select_vec], width=200)
col_2 = column([p1, p3])
col_3 = column([p2, p4])

curdoc().add_root(row(gridplot([[col_1, col_2, col_3]]), width=400))

#show(p2)

In [9]:
def histogram(df, col, bins=30):
    """
    Función para generar los intervalos y la cantidad de datos por intervalo
    
    Parámetros:
    -----------
    df: dataFrame de entrada, indice debe ser de tipo datetime
    col: columna guia para contar los intervalos
    bins: cantidad de intervalos
    
    Retorna:
    ---------
    hist: valores del histograma, cantidad de datos por intervalo.
    edges: Intervalos del histograma.
    """
    edges = []
    hist = []
    largura = max(data.index) - min(data.index)
    h = largura/bins
    aux = min(data.index)
    for i in range(100):
        edges.append(aux)
        hist.append(data.loc[aux:aux + h].count()[col])
        aux = aux + h
    edges.append(aux)
    
    return edges, hist

In [10]:
edges, hist = histogram(data,"location", bins=100)

In [11]:
p1 = figure(plot_height=400, plot_width=800, title="Quantidade de tweets por intervalo de tempo",
           tools="hover,pan,wheel_zoom,box_zoom,reset,save",x_axis_type='datetime')

p1.xaxis.axis_label = "dias registrados"
p1.yaxis.axis_label = "quantidade de tweets"
source1 = ColumnDataSource(dict(hist=hist,left=edges[:-1], right=edges[1:]))
p1.quad(top="hist", bottom=0, left="left", right="right", color = "blue", 
            line_color="white", alpha=.7, source=source1, legend="tweets")
p1.hover.tooltips = [("data inicial", "@left{%F %T}"),
                    ("data final", "@right{%F %T}"),
                    ("quantidade", "@hist")]

p1.hover.formatters = {'left': 'datetime', 'right': 'datetime'}
p1.hover.mode = "vline"
show(p1)

### Procesamiento del lenguaje natural
- Para procesar los tweets se utilizará la librería nltk https://www.nltk.org/ 

In [12]:
data.head()

,location,account,message
time,,,
2020-04-06 00:00:00,Weston,Opportunities2,"Take advantheeseage of theesehese One, theeser..."
2020-04-06 00:00:00,Southton,LazyBCouch,@WatchesThomasBird fork it you're back in he s...
2020-04-06 00:02:00,Southton,______3333_____,Two month left bedoreefore you loose this vibe...
2020-04-06 00:04:00,Broadview,RasoHorse49,"billeeeeer, i miss ytouou !"
2020-04-06 00:07:00,West Parton,CuriousPlateBobbie_Mallon,You obviously need to use rumble! #rumble #toW...


In [235]:
t_text = ""
data_explorer = data.loc[data.location=="Broadview"].message
for idx in data_explorer:
    t_text = t_text + " " + idx

In [236]:
t_text = t_text.lower()

In [237]:
nltk.text.FreqDist(nltk.word_tokenize(t_text))

FreqDist({'.': 1534, ':': 1378, 'the': 1299, 're': 1277, '!': 837, 'to': 784, 'i': 763, ',': 660, 'and': 652, 'a': 634, ...})

In [238]:
import re, unicodedata, inflect
import contractions

t_text = re.sub('\[[^]]*\]', '', t_text)
t_text = contractions.fix(t_text)
words = nltk.word_tokenize(t_text)

In [239]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in nltk.corpus.stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

words = normalize(words)

In [240]:
len(words)

16883

In [241]:
d = dict()
for i in words:
    if i in d:
        d[i] += 1
    else:
        d.update({i:1})

In [242]:
max(d.values())

279

In [243]:
import operator
max(d.items(), key=operator.itemgetter(1))

('something', 279)

In [245]:
sorted(d.items(), key=operator.itemgetter(1), reverse=True)

[('something', 279),
 ('anyone', 255),
 ('someone', 248),
 ('anything', 213),
 ('going', 93),
 ('power', 91),
 ('get', 82),
 ('see', 80),
 ('one', 79),
 ('like', 78),
 ('people', 75),
 ('good', 73),
 ('really', 70),
 ('time', 63),
 ('great', 62),
 ('got', 62),
 ('go', 57),
 ('need', 57),
 ('trying', 57),
 ('food', 55),
 ('know', 54),
 ('nuclear', 53),
 ('find', 52),
 ('would', 51),
 ('right', 51),
 ('lot', 50),
 ('city', 49),
 ('help', 48),
 ('movie', 46),
 ('best', 46),
 ('water', 46),
 ('day', 45),
 ('always', 45),
 ('even', 45),
 ('think', 42),
 ('heard', 42),
 ('safe', 41),
 ('back', 41),
 ('make', 41),
 ('want', 41),
 ('phone', 40),
 ('shelter', 40),
 ('much', 39),
 ('love', 38),
 ('friend', 38),
 ('house', 37),
 ('still', 37),
 ('could', 37),
 ('many', 37),
 ('himark', 37),
 ('place', 36),
 ('us', 36),
 ('hospital', 36),
 ('work', 35),
 ('said', 35),
 ('two', 34),
 ('new', 34),
 ('st', 34),
 ('better', 33),
 ('say', 33),
 ('fatalities', 32),
 ('tell', 31),
 ('getting', 30),
 ('co

In [130]:
np.unique(words) == "zoom"

array([False, False, False, ..., False,  True, False])

1

In [61]:
print(t_text.count("police"))
print(t_text.count("power"))
print(t_text.count("sewer"))
print(t_text.count("medical"))
print(t_text.count("rescue"))
print(t_text.count("help"))

4
193
25
5
35
152
